In [1]:
%pip install duckduckgo-search

StatementMeta(, 923191d3-1280-47c3-8482-e0734fef6bac, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 1.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.8 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.3
    Not uninstalling lxml at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-9ffd25d0-ca6f-40d8-89b1-11f207ecfb94
    Can't uninstall 'lxml'. No files were found to uninstall.
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Not uninstalling click at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-9ffd25d0-ca6f-40d8-89b1-11f207ecfb94
    Can't uninstall 'click'. No files were found to uninstall.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install -

In [9]:
import json
import os
from datetime import datetime
from duckduckgo_search import DDGS

# --- CONFIGURATION ---
# 1. Who are we watching?
COMPETITORS = ["Microsoft Fabric", "Databricks", "Snowflake Data Cloud"]
# 2. Where do we save the data? (The Bronze Path)
# /lakehouse/default/Files/ is the magic path in Fabric
SAVE_PATH = "/lakehouse/default/Files/Bronze/Landing"

# --- THE FUNCTION ---
def fetch_news_data():
    print("Starting News Ingestion...")
    
    # Create the folder if it doesn't exist
    os.makedirs(SAVE_PATH, exist_ok=True)
    
    current_time = datetime.now().isoformat()
    batch_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    all_results = []
    
    # Initialize the Search Engine
    with DDGS() as ddgs:
        for comp in COMPETITORS:
            print(f"Searching for: {comp}...")
            # specific search for 'news' results
            results = list(ddgs.news(keywords=comp, max_results=5))
            
            # Add metadata (Crucial for Engineering)
            for r in results:
                r['competitor_tag'] = comp
                r['ingestion_time'] = current_time
                r['batch_id'] = batch_id
                all_results.extend(results)
    
    # --- SAVE TO JSON ---
    filename = f"news_batch_{batch_id}.json"
    full_path = f"{SAVE_PATH}/{filename}"
    
    with open(full_path, "w") as f:
        json.dump(all_results, f)
        
    print(f"SUCCESS: Saved {len(all_results)} articles to:")
    print(full_path)

# --- EXECUTE ---
fetch_news_data()

StatementMeta(, 923191d3-1280-47c3-8482-e0734fef6bac, 16, Finished, Available, Finished)

/tmp/ipykernel_19845/1521197168.py:26: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Searching for: Databricks...
Searching for: Snowflake Data Cloud...
SUCCESS: Saved 75 articles to:
/lakehouse/default/Files/Bronze/Landing/news_batch_20251207_084727.json


In [6]:
import pandas as pd

df = pd.read_json('/lakehouse/default/Files/Bronze/Landing/news_batch_20251207_081942.json')
df.head()

StatementMeta(, 923191d3-1280-47c3-8482-e0734fef6bac, 13, Finished, Available, Finished)

,date,title,body,url,image,source,competitor_tag,ingestion_time,batch_id
0,2025-12-02 19:15:00+00:00,SAS' leading decision intelligence capabilitie...,What it is: SAS Decision Builder on Microsoft ...,https://www.tmcnet.com/usubmit/2025/12/02/1029...,https://mma.prnewswire.com/media/2836749/SAS_D...,TMCnet,Microsoft Fabric,2025-12-07 08:19:42.479283,20251207081942
1,2025-12-04 07:03:44+00:00,From complexity to clarity - why Microsoft Fab...,Microsoft Fabric provides a strategic response...,https://www.msn.com/en-za/news/other/from-comp...,https://www.itweb.co.za/static/pictures/2025/1...,ITWeb,Microsoft Fabric,2025-12-07 08:19:42.479283,20251207081942
2,2025-05-14 00:01:00+00:00,Informatica Unveils New Innovations with Micro...,"REDWOOD CITY, Calif., May 14, 2025--(BUSINESS ...",https://finance.yahoo.com/news/informatica-unv...,,Yahoo Finance,Microsoft Fabric,2025-12-07 08:19:42.479283,20251207081942
3,2024-04-17 00:01:00+00:00,Microsoft Kills 30-Day Data Retention Policy f...,Microsoft this week announced several usabilit...,https://redmondmag.com/articles/2024/04/17/mic...,,Redmond Magazine,Microsoft Fabric,2025-12-07 08:19:42.479283,20251207081942
4,2025-11-18 16:16:00+00:00,Microsoft updates database portfolio and adds ...,Microsoft Corp. today is introducing updates a...,https://siliconangle.com/2025/11/18/microsoft-...,,SiliconANGLE,Microsoft Fabric,2025-12-07 08:19:42.479283,20251207081942
